In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib notebook

<h1 align='center'><b>Natural Gas Power Plants</b></h1>
<p size=12>Sourced from EIA 923 "Page 1 Generation and Fuel Data"</p>

In [2]:
'''# Import excel file and pickle the sheet of interest
data = pd.read_excel('/home/alan/Desktop/Civil_Engineering/Thesis_Research/f923_2017/EIA923_Schedules_2_3_4_5_M_12_2017_Final_Revision.xlsx',
                    sheet_name = 'Page 1 Generation and Fuel Data',
                    index_col = None,
                    header = 5)
df = pd.DataFrame(data)
df.to_pickle('/home/alan/Desktop/Civil_Engineering/Thesis_Research/Pickle_Jar/Generation_and_FuelData.pkl')'''

"# Import excel file and pickle the sheet of interest\ndata = pd.read_excel('/home/alan/Desktop/Civil_Engineering/Thesis_Research/f923_2017/EIA923_Schedules_2_3_4_5_M_12_2017_Final_Revision.xlsx',\n                    sheet_name = 'Page 1 Generation and Fuel Data',\n                    index_col = None,\n                    header = 5)\ndf = pd.DataFrame(data)\ndf.to_pickle('/home/alan/Desktop/Civil_Engineering/Thesis_Research/Pickle_Jar/Generation_and_FuelData.pkl')"

In [3]:
df = pd.read_pickle('../pickleJar/Generation_and_FuelData.pkl')

In [4]:
df = df.loc[(df['Sector Name'] == 'Electric Utility') &
      (df['Reported\nFuel Type Code'] == 'NG')]
pickle.dump(df['Reported\nPrime Mover'].unique(), open('../pickleJar/PrimeMovers_NG', 'wb'))

In [5]:
e = pickle.load(open('../pickleJar/PlantIDs_NG.pkl', 'rb'))
e

array([    3,     7,     9, ..., 60647, 61508, 99999])

In [6]:
pickle.dump(df['Reported\nPrime Mover'].unique(), open('../pickleJar/PrimeMovers_NG','wb'))

In [7]:
f = pickle.load(open('../pickleJar/PrimeMovers.pkl', 'rb'))
f

array(['CA', 'CT', 'ST', 'GT', 'IC', 'CS', 'CE', 'FC'], dtype=object)

<h2 align='center'>NG Plants for Arizona Case Study</h2>

In [8]:
'''data = pd.read_excel('../EIA860/3_1_Generator_Y2017.xlsx',
                     sheet_name = 'Operable',
                     header = 1)
data.to_pickle('../pickleJar/3_1_Generator_Y2017_Operable.pkl')'''

"data = pd.read_excel('../EIA860/3_1_Generator_Y2017.xlsx',\n                     sheet_name = 'Operable',\n                     header = 1)\ndata.to_pickle('../pickleJar/3_1_Generator_Y2017_Operable.pkl')"

In [9]:
data = pd.read_pickle('../pickleJar/3_1_Generator_Y2017_Operable.pkl')

In [10]:
data['Technology'] = data['Technology'].fillna('invalid')

In [11]:
data['Technology'] = data['Technology'].apply(lambda x: x if 'Natural Gas' in x else 'Not Important')

In [12]:
data = data.loc[data['Technology'] != 'Not Important']

In [13]:
generators = data[['Plant Code',
                   'Plant Name',
                   'State',
                   'Technology',
                   'Prime Mover',
                   'Nameplate Capacity (MW)']]

In [14]:
AZ_Generators = generators.loc[generators['State'] == 'AZ']

In [15]:
'''
    EIA 860 Pipeline Data
'''
pipe = pd.read_pickle('../pickleJar/2__Plant_Y2017.pkl')
pipe = pipe[['Plant Code',
             'Latitude',
             'Longitude',
             'Natural Gas LDC Name',
             'Natural Gas Pipeline Name 1',
             'Natural Gas Pipeline Name 2',
             'Natural Gas Pipeline Name 3',
             'Pipeline Notes']]

<h3>EIA 860 Data</h3>

In [16]:
AZ_Plants = (AZ_Generators.
             groupby(['Plant Code', 'Plant Name'])['Nameplate Capacity (MW)'].
             sum().to_frame().reset_index(level='Plant Name'))

<h3>EIA 923 Data</h3>

In [17]:
Q = pd.read_pickle('../pickleJar/Fuel_Receipts_and_Costs.pkl')

S = pd.read_pickle('../pickleJar/Generation_and_FuelData.pkl')
S = S[['Plant Id',
       'Plant Name',
       'Sector Name']]
S = (S.groupby(['Plant Id',
               'Plant Name',
               'Sector Name'])['Sector Name'].
       count().to_frame().drop(columns=['Sector Name']).
       reset_index(level=['Plant Id', 'Plant Name', 'Sector Name']))

In [18]:
Q = pd.read_pickle('../pickleJar/Fuel_Receipts_and_Costs.pkl')

S = pd.read_pickle('../pickleJar/Generation_and_FuelData.pkl')
S = S[['Plant Id',
       'Plant Name',
       'Sector Name']]
S = (S.groupby(['Plant Id',
               'Plant Name',
               'Sector Name'])['Sector Name'].
       count().to_frame().drop(columns=['Sector Name']).
       reset_index(level=['Plant Id', 'Plant Name', 'Sector Name']))

Q = Q.loc[(Q['Plant State'].isin(['AZ']) &
           (Q['ENERGY_SOURCE'].isin(['NG'])))]
Q = Q[['Plant Id',
   'Plant Name',
   'Plant State',
   'FUEL_GROUP',
   'QUANTITY']]
Q = Q.merge(S,
        how='left',
        left_on='Plant Id',
        right_on='Plant Id',
        suffixes=('', '_y')).drop(columns=['Plant Name_y'])

In [19]:
Q_AZ = (Q.groupby(['Plant Id',
                  'Plant Name',
                  'Plant State',
                   'Sector Name'])['QUANTITY'].
        sum().to_frame().reset_index(level = ['Plant Name',
                                              'Plant State',
                                              'Sector Name']))

In [20]:
Z = Q_AZ.merge(AZ_Plants,
          how = 'left',
          left_on = Q_AZ.index,
          right_on = 'Plant Code',
          suffixes = ('','_y')).drop(columns=['Plant Name_y'])
complete = Z.merge(pipe,
                  how = 'left',
                  left_on = 'Plant Code',
                  right_on = 'Plant Code',
                  suffixes = ('','_y')).set_index('Plant Code')
complete

,Plant Name,Plant State,Sector Name,QUANTITY,Nameplate Capacity (MW),Latitude,Longitude,Natural Gas LDC Name,Natural Gas Pipeline Name 1,Natural Gas Pipeline Name 2,Natural Gas Pipeline Name 3,Pipeline Notes
Plant Code,,,,,,,,,,,,
113,Cholla,AZ,Electric Utility,1468,NaN,34.9394,-110.303,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
116,Ocotillo,AZ,Electric Utility,2851234,333.4,33.4225,-111.912,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
117,West Phoenix,AZ,Electric Utility,22705106,1207.4,33.4417,-112.158,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
120,Yucca,AZ,Electric Utility,3536334,327.3,32.7214,-114.711,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
126,H Wilson Sundt Generating Station,AZ,Electric Utility,8861675,558.5,32.16,-110.905,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
141,Agua Fria,AZ,Electric Utility,2371281,613.3,33.5561,-112.215,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
147,Kyrene,AZ,Electric Utility,7250836,573.7,33.3556,-111.935,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,NaN
160,Apache Station,AZ,Electric Utility,1825357,252.7,32.0603,-109.893,NaN,EL PASO NATURAL GAS COMPANY LLC,NaN,NaN,KINDER MORGAN BOUGHT OUT EPNG
8068,Santan,AZ,Electric Utility,27458965,1326.0,33.3325,-111.75,NaN,NaN,Other - Please explain in pipeline notes below.,NaN,Kinder Morgan-El Paso Natural Gas LLC.


In [21]:
complete.to_excel('../Output_Excel/Arizona_Info.xlsx')
complete.to_pickle('../pickleJar/Arizona_Info.pkl')

<h3>Plants in EIA 860 that are not
    in EIA 923</h3>

In [22]:
AZ_Plants.loc[AZ_Plants.
               index.
               isin(np.setdiff1d(AZ_Plants.index.values, Q_AZ.index.values))]

,Plant Name,Nameplate Capacity (MW)
Plant Code,,
118.0,Saguaro,184.5
124.0,Demoss Petrie,85.0
6088.0,North Loop,107.8
6515.0,Valencia,108.0
54594.0,University of Arizona - Biosphere 2,1.6
54694.0,Yuma Cogeneration Associates,62.6
55177.0,South Point Energy Center,708.0
56229.0,Cogeneration 1,9.0
56482.0,Black Mountain Generating Station,121.0


In [23]:
Q_AZ.loc[Q_AZ.
         index.
         isin(np.setdiff1d(AZ_Plants.index.values, Q_AZ.index.values))]

,Plant Name,Plant State,Sector Name,QUANTITY
Plant Id,,,,


<h3>Conclusion</h3>
<p>
    The EIA 860 data is more representative of the operational plants in Arizona than the<br>
    EIA 923 data.
</p><br>
<h5>Solution:</h5>
<p>
   Will need to either revise the current basic model, or find a new source of fuel<br>
    quantity data.
</p>

In [24]:
R = pd.read_pickle('../pickleJar/Fuel_Receipts_and_Costs.pkl')

In [25]:
R.loc[R['Plant Id'] == 118]

,YEAR,MONTH,Plant Id,Plant Name,Plant State,Purchase Type,Contract\nExpiration Date,ENERGY_SOURCE,FUEL_GROUP,Coalmine\nType,...,Regulated,Operator Name,Operator Id,Reporting\nFrequency,Primary Transportation Mode,Secondary Transportation Mode,Natural Gas Supply Contract Type,Natural Gas Delivery Contract Type,Moisture\nContent,Chlorine\nContent


<h3>Evaluate</h3>
<p>
    Sum the total NG consumed for the plants listed in the EIA 923 and compare<br>
    to the total NG supplied to Arizona
</p>

In [26]:
Q_AZ['QUANTITY'].sum()/1e3

224020.737

In [27]:
Q_AZ.loc[Q_AZ['Sector Name'] == 'Electric Utility']['QUANTITY'].sum()/1e3

159621.796